In [5]:
import cv2
import numpy as np
import math

In [ ]:
def get_run_length_encoding(image):
    i = 0
    skip = 0
    stream = []    
    bitstream = ""
    image = image.astype(int)
    while i < image.shape[0]:
        if image[i] != 0:            
            stream.append((image[i],skip))
            bitstream = bitstream + str(image[i])+ " " +str(skip)+ " "
            skip = 0
        else:
            skip = skip + 1
        i = i + 1

    return bitstream

# Размер блоков
block_size = 8

# Матрциа Квантования
QUANTIZATION_MAT = np.array([[16,11,10,16,24,40,51,61],[12,12,14,19,26,58,60,55],[14,13,16,24,40,57,69,56 ],[14,17,22,29,51,87,80,62],[18,22,37,56,68,109,103,77],[24,35,55,64,81,104,113,92],[49,64,78,87,103,121,120,101],[72,92,95,98,112,100,103,99]])


img = cv2.imread('14.jpg', cv2.IMREAD_GRAYSCALE)

[h , w] = img.shape
height = h
width = w
h = np.float32(h) 
w = np.float32(w) 

nbh = math.ceil(h/block_size)
nbh = np.int32(nbh)

nbw = math.ceil(w/block_size)
nbw = np.int32(nbw)

In [ ]:
# Подстраиваем размер изображения под выбранный размер блоков, дабы избежать проблем т.к может не делиться

H =  block_size * nbh

W =  block_size * nbw

padded_img = np.zeros((H,W))

padded_img[0:height,0:width] = img[0:height,0:width]

cv2.imwrite('uncompressed.jpg', np.uint8(padded_img))


# Начинаем делить изображение на блоки, и применяем коэффиценты матрицы в зиг-заг порядке
for i in range(nbh):
        row_ind_1 = i*block_size                
        row_ind_2 = row_ind_1+block_size
        
        for j in range(nbw):
            col_ind_1 = j*block_size                       
            col_ind_2 = col_ind_1+block_size
                        
            block = padded_img[ row_ind_1 : row_ind_2 , col_ind_1 : col_ind_2 ]                      
            DCT = cv2.dct(block)            
            DCT_normalized = np.divide(DCT,QUANTIZATION_MAT).astype(int)            
            reordered = zigzag(DCT_normalized)
            reshaped= np.reshape(reordered, (block_size, block_size)) 
            padded_img[row_ind_1 : row_ind_2 , col_ind_1 : col_ind_2] = reshaped                        

cv2.imshow('encoded image', np.uint8(padded_img))

arranged = padded_img.flatten()

# записываем все в тхт файл, чтобы можно было бы раскодировать в дальнейшем
bitstream = get_run_length_encoding(arranged)

bitstream = str(padded_img.shape[0]) + " " + str(padded_img.shape[1]) + " " + bitstream + ";"

file1 = open("image.txt","w")
file1.write(bitstream)
file1.close()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
#декодинг
QUANTIZATION_MAT = np.array([[16,11,10,16,24,40,51,61],[12,12,14,19,26,58,60,55],[14,13,16,24,40,57,69,56 ],[14,17,22,29,51,87,80,62],[18,22,37,56,68,109,103,77],[24,35,55,64,81,104,113,92],[49,64,78,87,103,121,120,101],[72,92,95,98,112,100,103,99]])
block_size = 8
# читаем наш тхт файл
with open('image.txt', 'r') as myfile:
    image=myfile.read()
details = image.split()
h = int(''.join(filter(str.isdigit, details[0])))
w = int(''.join(filter(str.isdigit, details[1])))
array = np.zeros(h*w).astype(int)

k = 0
i = 2
x = 0
j = 0

True

In [ ]:
# Новый размер изображения

while k < array.shape[0]:
    if(details[i] == ';'):
        break
    if "-" not in details[i]:
        array[k] = int(''.join(filter(str.isdigit, details[i])))        
    else:
        array[k] = -1*int(''.join(filter(str.isdigit, details[i])))        
    if(i+3 < len(details)):
        j = int(''.join(filter(str.isdigit, details[i+3])))
    if j == 0:
        k = k + 1
    else:                
        k = k + j + 1        
    i = i + 2
array = np.reshape(array,(h,w))
i = 0
j = 0
k = 0
padded_img = np.zeros((h,w))

while i < h:
    j = 0
    while j < w:        
        temp_stream = array[i:i+8,j:j+8]                
        block = inverse_zigzag(temp_stream.flatten(), int(block_size),int(block_size))            
        de_quantized = np.multiply(block,QUANTIZATION_MAT)                
        padded_img[i:i+8,j:j+8] = cv2.idct(de_quantized)        
        j = j + 8        
    i = i + 8

# 8бит
padded_img[padded_img > 255] = 255
padded_img[padded_img < 0] = 0
cv2.imwrite("compressed_image.jpg",np.uint8(padded_img))


In [9]:
def entropy(img):
    frcy = np.array([0 for i in range(256)])
    for row in img:
        for px in row:
            frcy[px] += 1
            
    n = len(img) * len(img[0])
    frcy = frcy / n
    ent = -np.sum([p * np.log2(p) for p in frcy if p != 0])
    return ent
def psnr(img1, img2):
    rmse = np.sqrt(np.sum(np.power(img1 - img2, 2)) / img1.shape[0] / img1.shape[1])
    if rmse == 0:
        return 100
    pxmax = 255.0
    return 20 * math.log10(pxmax / math.sqrt(rmse))

In [10]:
a = cv2.imread('14.jpg')
c = np.uint8(padded_img)
entropy(a)

7.634483223644974

In [11]:
entropy(a) # исходник

7.634483223644974

In [12]:
entropy(c) # итог

7.5159972039605698

In [26]:
d = cv2.imread("compressed_image.jpg")
psnr(img, c) # сравниваем обработанный и исходный

42.39518804707859

In [25]:
psnr(a,d)# png сравниваем исходные и сохранившую

40.010095149422035

In [ ]:
import numpy as np

def zigzag(input):
    #initializing the variables
    #----------------------------------
    h = 0
    v = 0

    vmin = 0
    hmin = 0

    vmax = input.shape[0]
    hmax = input.shape[1]
    
    #print(vmax ,hmax )

    i = 0

    output = np.zeros(( vmax * hmax))
    #----------------------------------

    while ((v < vmax) and (h < hmax)):
    	
        if ((h + v) % 2) == 0:                 # going up
            
            if (v == vmin):
            	#print(1)
                output[i] = input[v, h]        # if we got to the first line

                if (h == hmax):
                    v = v + 1
                else:
                    h = h + 1                        

                i = i + 1

            elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
            	#print(2)
            	output[i] = input[v, h] 
            	v = v + 1
            	i = i + 1

            elif ((v > vmin) and (h < hmax -1 )):    # all other cases
            	#print(3)
            	output[i] = input[v, h] 
            	v = v - 1
            	h = h + 1
            	i = i + 1

        
        else:                                    # going down

        	if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
        		#print(4)
        		output[i] = input[v, h] 
        		h = h + 1
        		i = i + 1
        
        	elif (h == hmin):                  # if we got to the first column
        		#print(5)
        		output[i] = input[v, h] 

        		if (v == vmax -1):
        			h = h + 1
        		else:
        			v = v + 1

        		i = i + 1

        	elif ((v < vmax -1) and (h > hmin)):     # all other cases
        		#print(6)
        		output[i] = input[v, h] 
        		v = v + 1
        		h = h - 1
        		i = i + 1




        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
        	#print(7)        	
        	output[i] = input[v, h] 
        	break

    #print ('v:',v,', h:',h,', i:',i)
    return output




# Inverse zigzag scan of a matrix
# Arguments are: a 1-by-m*n array, 
# where m & n are vertical & horizontal sizes of an output matrix.
# Function returns a two-dimensional matrix of defined sizes,
# consisting of input array items gathered by a zigzag method.



def inverse_zigzag(input, vmax, hmax):
	
	#print input.shape

	# initializing the variables
	#----------------------------------
	h = 0
	v = 0

	vmin = 0
	hmin = 0

	output = np.zeros((vmax, hmax))

	i = 0
    #----------------------------------

	while ((v < vmax) and (h < hmax)): 
		#print ('v:',v,', h:',h,', i:',i)   	
		if ((h + v) % 2) == 0:                 # going up
            
			if (v == vmin):
				#print(1)
				
				output[v, h] = input[i]        # if we got to the first line

				if (h == hmax):
					v = v + 1
				else:
					h = h + 1                        

				i = i + 1

			elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
				#print(2)
				output[v, h] = input[i] 
				v = v + 1
				i = i + 1

			elif ((v > vmin) and (h < hmax -1 )):    # all other cases
				#print(3)
				output[v, h] = input[i] 
				v = v - 1
				h = h + 1
				i = i + 1

        
		else:                                    # going down

			if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
				#print(4)
				output[v, h] = input[i] 
				h = h + 1
				i = i + 1
        
			elif (h == hmin):                  # if we got to the first column
				#print(5)
				output[v, h] = input[i] 
				if (v == vmax -1):
					h = h + 1
				else:
					v = v + 1
				i = i + 1
        		        		
			elif((v < vmax -1) and (h > hmin)):     # all other cases
				output[v, h] = input[i] 
				v = v + 1
				h = h - 1
				i = i + 1




		if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
			#print(7)        	
			output[v, h] = input[i] 
			break


	return output